In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import mean_squared_error
from math import *
import matplotlib.pyplot as plt
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()
import warnings
warnings.filterwarnings('ignore')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "15.0.1" 2020-10-20; OpenJDK Runtime Environment (build 15.0.1+9-18); OpenJDK 64-Bit Server VM (build 15.0.1+9-18, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/z_/w3xlnh2d73q7rhjpcns83fn40000gn/T/tmpjmdfnr5y
  JVM stdout: /var/folders/z_/w3xlnh2d73q7rhjpcns83fn40000gn/T/tmpjmdfnr5y/h2o_sreeya_started_from_python.out
  JVM stderr: /var/folders/z_/w3xlnh2d73q7rhjpcns83fn40000gn/T/tmpjmdfnr5y/h2o_sreeya_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Jakarta
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,3 months and 14 days !!!
H2O_cluster_name:,H2O_from_python_sreeya_5yef28
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
testing_data = pd.read_excel("testingt.xlsx")
testing_data1 = pd.read_excel("testingt.xlsx")
data = pd.read_excel("ff_final_dataset.xlsx")
data1 = pd.read_excel("ff_final_dataset.xlsx")
testing_data.drop("Unnamed: 0", axis=1, inplace=True)
testing_data.drop("Date", axis=1, inplace=True)
testing_data.drop("Wellbore ID", axis=1, inplace=True)
testing_data1.drop("Wellbore ID", axis=1, inplace=True)
testing_data1 = testing_data1.dropna()
testing_data = testing_data.dropna()
data = data.dropna()
data1 = data1.dropna()

In [3]:
data_pred = h2o.H2OFrame(testing_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
data_f = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train, valid = data_f.split_frame(ratios = [.9])

In [6]:
x = train.columns
y = "Oil (m3)"
x.remove(y)
x.remove('Gas (m3)')

# Oil Prediction

# Gradient Boosting

In [7]:
pros_gbm = H2OGradientBoostingEstimator()
pros_gbm.train(x=x, y=y, training_frame=train, validation_frame=valid)
perf = pros_gbm.model_performance()
pred = pros_gbm.predict(data_pred)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [8]:
pr = h2o.as_list(pred)

In [9]:
rmse = sqrt(mean_squared_error(pr['predict'], testing_data['Oil (m3)'])) 
print("RMSE : ", rmse)

RMSE :  303.2566303526326


# Random Forest

In [10]:
s_drf = H2ORandomForestEstimator()
s_drf.train(x=x, y=y, training_frame=train, validation_frame=valid)
perf1 = s_drf.model_performance()
pred1 = s_drf.predict(data_pred)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [11]:
pred_on_train = s_drf.predict(data_f)
rf_train = pred_on_train.as_data_frame()

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [12]:
pr1 = pred1.as_data_frame()

In [13]:
rmse = sqrt(mean_squared_error(pr1['predict'], testing_data['Oil (m3)'])) 
print("RMSE : ", rmse)

RMSE :  372.56657324620994


# Gas Prediction

In [14]:
x = train.columns
y = "Gas (m3)"
x.remove(y)
x.remove('Oil (m3)')

# Gradient Boosting

In [15]:
pros_gbm = H2OGradientBoostingEstimator()
pros_gbm.train(x=x, y=y, training_frame=train, validation_frame=valid)
perf = pros_gbm.model_performance()
pred = pros_gbm.predict(data_pred)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [16]:
data_as_list = h2o.as_list(pred, use_pandas=True)
pr = h2o.as_list(pred)

In [17]:
rmse = sqrt(mean_squared_error(pr['predict'], testing_data['Gas (m3)'])) 
print("RMSE : ", rmse)

RMSE :  139915.4840198639


# Random Forest

In [18]:
s_drf = H2ORandomForestEstimator()
s_drf.train(x=x, y=y, training_frame=train, validation_frame=valid)
perf1 = s_drf.model_performance()
pred1 = s_drf.predict(data_pred)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [19]:
data_as_list = h2o.as_list(pred1, use_pandas=False)
pr1 = h2o.as_list(pred1)

In [20]:
rmse = sqrt(mean_squared_error(pr1['predict'], testing_data['Gas (m3)'])) 
print("RMSE : ", rmse)

RMSE :  189661.67569354054
